In [1]:
import numpy as np
import math

# Probabilistic Neural Network with 4 layers
class PNN(object):
    def __init__(self):
        self.L2 = []    # Layer 2 that holds the patterns
        print('Empty PNN created.')

    def train(self, X, y, p=2):
        self.n_ = X.shape[1]  # num of features
        self.p_ = p           # num of classes
        
        # Layer 2 (Pattern): Set up empty lists for each class
        for k in range(self.p_):
            self.L2.append([])    # Using Python's basic lists because ndarray cannot append empty arrays
                                  # Also perhaps we might have to input different data types

        # Enter patterns into Layer 2
        for i in range(X.shape[0]):
            self.L2[y[i]].append(X[i])

        print('PNN with', self.p_, 'classes trained.')

    def crossValidate(self, X, y, sigma=0.5):
        result = self.predict(X, sigma)
        num_correct = sum(result[:, 0] == y)

        print('Cross validation accuracy with sigma', sigma, ':', num_correct/len(y) * 100, '%')

    def predict(self, X, sigma=0.5):
        self.sigma_ = sigma    # smoothing parameter, not standard deviation

        m = X.shape[0]
        accL3 = np.zeros((m, self.p_))
        accL4 = np.zeros(m)

        # Layer 1 (Input): x
        for i in range(m):
            x = X[i]

            # Layer 3 (Averaging): for each class
            self.L3_ = np.zeros(self.p_)
            for k in range(self.p_):
                for ki in range(len(self.L2[k])):
                    self.L3_[k] += self._activation(x, self.L2[k][ki])
                self.L3_[k] /= len(self.L2[k])
                
                # Multiply constant
                self.L3_[k] *= (math.sqrt(2*math.pi) * self.sigma_)**(- self.n_)
                accL3[i][k] = self.L3_[k]

            # Layer 4 (Output/Decision): Maxing
            self.L4_ = self.L3_.argmax()
            accL4[i] = self.L4_

        return np.column_stack((accL4, accL3))

    def _activation(self, x, w):
        diff = x - w
        return math.exp( - np.dot(diff, diff) / (2 * self.sigma_**2) )


# Normalize to unit length: [0, 1]
# X must be ndarray
def Normalize(X):
    x_max = X.max(axis=0)
    x_min = X.min(axis=0)
    return (X - x_min) / (x_max - x_min)

In [2]:
import pandas as pd

df = pd.read_csv('https://archive.ics.uci.edu/ml/machine-learning-databases/iris/iris.data', header=None)
df.tail()

,0,1,2,3,4
145,6.7,3.0,5.2,2.3,Iris-virginica
146,6.3,2.5,5.0,1.9,Iris-virginica
147,6.5,3.0,5.2,2.0,Iris-virginica
148,6.2,3.4,5.4,2.3,Iris-virginica
149,5.9,3.0,5.1,1.8,Iris-virginica


In [3]:
X = df.iloc[:, :4].values

y = df.iloc[:, 4].values
y[:50] = 0
y[50:100] = 1
y[100:] = 2

X_N = Normalize(X)

# Training data
X_tr = np.row_stack((X_N[0:40], X_N[50:90], X_N[100:140]))
y_tr = np.append(np.append(y[0:40], y[50:90]), y[100:140])

# Cross validation data (may be same as training data for a PNN)
X_cv = X_tr
y_cv = y_tr

# Test data
X_tt = np.row_stack((X_N[40:50], X_N[90:100], X_N[140:150]))
y_tt = np.append(np.append(y[40:50], y[90:100]), y[140:150])

In [4]:
pnn = PNN()

Empty PNN created.


In [5]:
pnn.train(X_tr, y_tr, p=3)

PNN with 3 classes trained.


In [6]:
n_iter = 10
for s in range(n_iter):
    pnn.crossValidate(X_cv, y_cv, sigma=(s+1)/n_iter)

Cross validation accuracy with sigma 0.1 : 97.5 %
Cross validation accuracy with sigma 0.2 : 95.0 %
Cross validation accuracy with sigma 0.3 : 93.3333333333 %
Cross validation accuracy with sigma 0.4 : 90.8333333333 %
Cross validation accuracy with sigma 0.5 : 90.0 %
Cross validation accuracy with sigma 0.6 : 90.0 %
Cross validation accuracy with sigma 0.7 : 90.0 %
Cross validation accuracy with sigma 0.8 : 90.0 %
Cross validation accuracy with sigma 0.9 : 90.0 %
Cross validation accuracy with sigma 1.0 : 89.1666666667 %


In [7]:
result = pnn.predict(X_tt, sigma=0.4)
print(result)

[[ 0.          0.88633248  0.13599766  0.02351808]
 [ 0.          0.46479177  0.14495074  0.02016   ]
 [ 0.          0.80095261  0.10984012  0.01576763]
 [ 0.          0.82714536  0.20680046  0.04535853]
 [ 0.          0.80897973  0.15480814  0.03324777]
 [ 0.          0.81192232  0.17563272  0.02844401]
 [ 0.          0.83242541  0.10918628  0.01968301]
 [ 0.          0.8404094   0.12998154  0.01961307]
 [ 0.          0.84849294  0.1235897   0.02240711]
 [ 0.          0.88668957  0.14796587  0.02406152]
 [ 1.          0.16836468  0.77926535  0.41553547]
 [ 1.          0.12464073  0.82457446  0.59798557]
 [ 1.          0.18650972  0.80989642  0.42198754]
 [ 1.          0.25966721  0.54677625  0.18660435]
 [ 1.          0.17580319  0.81181204  0.45511965]
 [ 1.          0.22726354  0.79803544  0.44095711]
 [ 1.          0.19387981  0.82062785  0.47731591]
 [ 1.          0.14853016  0.83557199  0.54338924]
 [ 1.          0.31641467  0.58704309  0.20913101]
 [ 1.          0.1916662   0.82

In [8]:
from numpy.random import seed
random_state = None
seed(random_state)
r = np.random.permutation(len(y))
X_N2, y2 = X_N[r], y[r]

# Training data
X_tr2 = X_N2[:120]
y_tr2 = y2[:120]

# Cross validation data (may be same as training data for a PNN)
X_cv2 = X_tr2
y_cv2 = y_tr2

# Test data
X_tt2 = X_tt
y_tt2 = y_tt

pnn2 = PNN()
pnn2.train(X_tr2, y_tr2, p=3)

Empty PNN created.
PNN with 3 classes trained.


In [9]:
n_iter = 10
for s in range(n_iter):
    pnn2.crossValidate(X_cv2, y_cv2, sigma=(s+1)/n_iter)

Cross validation accuracy with sigma 0.1 : 98.3333333333 %
Cross validation accuracy with sigma 0.2 : 96.6666666667 %
Cross validation accuracy with sigma 0.3 : 96.6666666667 %
Cross validation accuracy with sigma 0.4 : 95.8333333333 %
Cross validation accuracy with sigma 0.5 : 95.0 %
Cross validation accuracy with sigma 0.6 : 95.0 %
Cross validation accuracy with sigma 0.7 : 94.1666666667 %
Cross validation accuracy with sigma 0.8 : 91.6666666667 %
Cross validation accuracy with sigma 0.9 : 91.6666666667 %
Cross validation accuracy with sigma 1.0 : 91.6666666667 %


In [10]:
result2 = pnn2.predict(X_tt2, sigma=0.4)
print(result2)

[[ 0.          0.87661707  0.14691445  0.02337442]
 [ 0.          0.47449517  0.15738993  0.01859058]
 [ 0.          0.79531441  0.11974304  0.01539947]
 [ 0.          0.8232929   0.22107534  0.04585834]
 [ 0.          0.80245188  0.16512243  0.03358445]
 [ 0.          0.80882536  0.19070096  0.02748838]
 [ 0.          0.82196159  0.11739903  0.01968104]
 [ 0.          0.83400682  0.14148625  0.01910409]
 [ 0.          0.8368278   0.13277668  0.02224439]
 [ 0.          0.87723771  0.16046503  0.0234766 ]
 [ 1.          0.17255101  0.79936287  0.40797532]
 [ 1.          0.12696221  0.82528302  0.60018814]
 [ 1.          0.19040991  0.82968111  0.41438739]
 [ 1.          0.26823936  0.57516137  0.17926259]
 [ 1.          0.17996934  0.82934561  0.45236311]
 [ 1.          0.23099171  0.81416325  0.44079162]
 [ 1.          0.19769726  0.83516511  0.47822004]
 [ 1.          0.15081376  0.84115469  0.54065539]
 [ 1.          0.32478167  0.61630698  0.20512423]
 [ 1.          0.19564519  0.84